# Neural networks, Checkpoint 

Faculty: FIIT <br>
Academic year: 2019/2020 <br>
Authors: Andrej Gáfrik, Martin Grega <br>


Source dataset: https://figshare.com/articles/Wikipedia_Detox_Data/4054689,

In [2]:
# !pip install pandas
# !pip install nltk
# !pip install tokenizer
#!pip uninstall keras
#!pip install keras --upgrade
#!pip install sklearn
#!pip install h5py

     |████████████████████████████████| 7.0MB 2.4MB/s eta 0:00:01     |█████▌                          | 1.2MB 2.4MB/s eta 0:00:03
     |████████████████████████████████| 25.2MB 6.8MB/s eta 0:00:011    |██▏                             | 1.8MB 10.1MB/s eta 0:00:03     |█████████████████████████████▏  | 23.0MB 6.8MB/s eta 0:00:01
     |████████████████████████████████| 296kB 12.0MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=2cedd9a8b33c4185fb895c6cd38b1b84a70ba8d53cc887cac54a095516cb8767
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import tensorflow.keras as keras

from nltk.stem import SnowballStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from numpy import array, asarray,zeros
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense,Dropout,SimpleRNN, LSTM, Bidirectional, Masking
from tensorflow.keras.initializers import Constant
from sklearn.utils import class_weight
from datetime import datetime

#from numpy import asarray
#from numpy import zeros
#from tensorflow.keras.layers import Embedding,Dense,SimpleRNN, LSTM, Bidirectional, Masking
#from tensorflow.keras.layers import Embedding,Dense,SimpleRNN, Masking, LSTM

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
comments = pd.read_csv('../../data/attack_annotated_comments.tsv', sep='\t')
labels = pd.read_csv('../../data/attack_annotations.tsv', sep='\t')

In [9]:
comments = comments.drop(columns=['year','logged_in','ns','sample','split'])

In [10]:
comments

,rev_id,comment
0,37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...
1,44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...
2,49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s..."
3,89320,"Next, maybe you could work on being less cond..."
4,93890,This page will need disambiguation.
...,...,...
115859,699848324,`NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENThese ...
115860,699851288,NEWLINE_TOKENNEWLINE_TOKENThe Institute for Hi...
115861,699857133,NEWLINE_TOKEN:The way you're trying to describ...
115862,699891012,NEWLINE_TOKENNEWLINE_TOKEN== Warning ==NEWLINE...


In [11]:
labels = labels.drop(columns=['worker_id','quoting_attack','recipient_attack','third_party_attack','other_attack'])

In [12]:
labels



,rev_id,attack
0,37675,0.0
1,37675,0.0
2,37675,0.0
3,37675,0.0
4,37675,0.0
...,...,...
1365212,699897151,0.0
1365213,699897151,0.0
1365214,699897151,0.0
1365215,699897151,0.0


In [13]:
res = labels.groupby('rev_id')['attack'].mean().reset_index()

In [14]:
data = comments.join(res.set_index('rev_id'), on='rev_id')
data['attack'] = data['attack'].round(0)

In [15]:
data

,rev_id,comment,attack
0,37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,0.0
1,44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,0.0
2,49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...",0.0
3,89320,"Next, maybe you could work on being less cond...",0.0
4,93890,This page will need disambiguation.,0.0
...,...,...,...
115859,699848324,`NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENThese ...,0.0
115860,699851288,NEWLINE_TOKENNEWLINE_TOKENThe Institute for Hi...,0.0
115861,699857133,NEWLINE_TOKEN:The way you're trying to describ...,0.0
115862,699891012,NEWLINE_TOKENNEWLINE_TOKEN== Warning ==NEWLINE...,0.0


# DATA analysis

In [17]:
data['attack'].value_counts()

0.0    102274
1.0     13590
Name: attack, dtype: int64

In [23]:
#No null values
data.info()
data.isnull().values.any()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115864 entries, 0 to 115863
Data columns (total 3 columns):
rev_id     115864 non-null int64
comment    115864 non-null object
attack     115864 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.7+ MB


False

In [24]:
max(data['comment'],key=len)

'NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOK

Problems in data:
 - tabular, new line and qoutes were replaced with tab_token, new_line token and `
 - punctuation
 - more spaces between words
 - empty strings
 - abbreviations like i will = i'll , didn't = did not, ...
 - unbalanced data
 - later we will also check length of comments
 - repetition of words

In [25]:
# In out data are newlines, tabulators and quotions marks replaced, so we need to delete this replacement
data['comment'] = data['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN",""))
data['comment'] = data['comment'].apply(lambda x: x.replace("TAB_TOKEN",""))
data['comment'] = data['comment'].apply(lambda x: x.replace("``",'"'))

In [26]:
# Source: https://www.kaggle.com/prashantkikani/pooled-gru-with-preprocessing
repl = {
    "&lt;3": " good ",
    ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "'s" : " is",
    "'s" : " is",
    "`d": " would",
    "that's" : "that is",
    "weren't" : "were not",
    "e.g" : "eg",
}

In [27]:
#splitting comments by spaces
comments = data['comment'].apply(lambda x: x.split())

In [28]:
abbr = [i for i in repl.keys()]
result = []
for index in range(len(comments)):
    new_comment = ""
    for word in comments[index]:
        word = word.lower()
        word = re.sub(r"n't", " not", word)
        word = re.sub(r"\ 's", " not", word)
        word = re.sub(r"\ 've", " not", word)
        word = re.sub(r"\ 'd", " not", word)
        word = re.sub(r"\ ll", " not", word)
        if re.search("^http.+|^www.+",word): # deleting links
            continue
        elif word in abbr:
            new_comment += repl[word]
            new_comment += " "
            word = repl[word]
        elif re.search("[^a-zA-Z ]+",word): # only alphabet
            new_comment += re.sub("[^a-zA-Z ]+"," ",word)
            new_comment += " "
        else:
            new_comment += word  
            new_comment += " "
            
    new_comment = " ".join(new_comment.split())        
   # print("Old = ", comments[index])
   # print("New = ", new_comment)
    result.append(new_comment)
data["comment"] = result

In [29]:
# replacing empty strings with nan
data['comment'].replace('', np.nan, inplace=True)

In [30]:
# checking nan values on data
data.info()
data.isnull().values.any()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115864 entries, 0 to 115863
Data columns (total 3 columns):
rev_id     115864 non-null int64
comment    115825 non-null object
attack     115864 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.7+ MB


True

In [31]:
# We now have null rows, because there were empty strings so we drop them
data.dropna(subset=['comment'], inplace=True)
data = data.reset_index(drop=True)

# Stemming

In [33]:
def stemmer(comment):
    stemmer = SnowballStemmer('english')
    words = comment.split()
    stemmed = [stemmer.stem(word) for word in words]
    stemmed
    return " ".join(stemmed)

data['stemmed_comments'] = data['comment'].map(lambda x: stemmer(x))

In [37]:
data['number_of_words'] = data.stemmed_comments.apply(lambda x: len(x.split()))

In [38]:
data

,rev_id,comment,attack,stemmed_comments,number_of_words
0,37675,this is not creative those are the dictionary ...,0.0,this is not creativ those are the dictionari d...,403
1,44816,the term standard model is itself less npov th...,0.0,the term standard model is itself less npov th...,123
2,49851,true or false the situation as of march was su...,0.0,true or fals the situat as of march was such a...,61
3,89320,next maybe you could work on being less condes...,0.0,next mayb you could work on be less condescend...,75
4,93890,this page will need disambiguation,0.0,this page will need disambigu,5
...,...,...,...,...,...
115820,699848324,these sources do not exactly exude a sense of ...,0.0,these sourc do not exact exud a sens of impart...,113
115821,699851288,the institute for historical review is a peer ...,0.0,the institut for histor review is a peer revie...,80
115822,699857133,the way you re trying to describe it in this a...,0.0,the way you re tri to describ it in this artic...,34
115823,699891012,warning there is clearly a protectionist regim...,0.0,warn there is clear a protectionist regim go o...,135


# Additional data cleaning

In [39]:
# Here we delete comments longer than 300 words, because neural network was training too long
data = data.loc[data['number_of_words'] <= 300]

In [40]:
data

,rev_id,comment,attack,stemmed_comments,number_of_words
1,44816,the term standard model is itself less npov th...,0.0,the term standard model is itself less npov th...,123
2,49851,true or false the situation as of march was su...,0.0,true or fals the situat as of march was such a...,61
3,89320,next maybe you could work on being less condes...,0.0,next mayb you could work on be less condescend...,75
4,93890,this page will need disambiguation,0.0,this page will need disambigu,5
5,102817,important note for all sysops there is a bug i...,0.0,import note for all sysop there is a bug in th...,47
...,...,...,...,...,...
115820,699848324,these sources do not exactly exude a sense of ...,0.0,these sourc do not exact exud a sens of impart...,113
115821,699851288,the institute for historical review is a peer ...,0.0,the institut for histor review is a peer revie...,80
115822,699857133,the way you re trying to describe it in this a...,0.0,the way you re tri to describ it in this artic...,34
115823,699891012,warning there is clearly a protectionist regim...,0.0,warn there is clear a protectionist regim go o...,135


In [43]:
data['attack'].value_counts()
# we deleted 384 comments with hate speech and 4k normal comments

0.0    98887
1.0    13206
Name: attack, dtype: int64

In [44]:
# saving cleaned data
data.to_csv('../../data/preprocessed_data_1.csv', encoding='utf-8')

# Tokenizing

In [45]:
data= pd.read_csv('../../data/preprocessed_data_1.csv', index_col=0)

In [46]:
tokenizer = Tokenizer()

# Training of our tokenizer
tokenizer.fit_on_texts(data['stemmed_comments'])
#tokenizer.fit_on_texts(data['comment'])

# Number of unique words in our data
vocab_len = len(tokenizer.word_index) + 1

#converting comment into numeric form, each unique word has a number, so comment will be rewrite into numbers
embedded = tokenizer.texts_to_sequences(data['stemmed_comments'])

In [47]:
longest_sentence_len = len(max(embedded, key= len))

#Adding zeroes to end of eat embeded sentece to len of the longest sentence
padded_comments = pad_sequences(embedded, longest_sentence_len, padding='post')

In [48]:
# Train and test data
size_train = round(len(padded_comments)* 0.8)

train_padded = padded_comments[:size_train]
train_attack =asarray(data['attack'][:size_train], dtype='int32')

test_padded = padded_comments[size_train + 1:]
test_attack = asarray(data['attack'][size_train + 1:], dtype='int32')

In [49]:
unique, counts = np.unique(train_attack, return_counts=True)
dict(zip(unique, counts))

{0: 78913, 1: 10761}

In [50]:
unique, counts = np.unique(test_attack, return_counts=True)
dict(zip(unique, counts))

{0: 19973, 1: 2445}

# Glove embeddings

In [51]:
glove = open('../../data/glove.6B.100d.txt', encoding="utf8")

In [52]:
# Preparing glove dictonary
glove_dict = dict()

#according to glove file, first is word and next are vectors
for line in glove:
    splitted = line.split()
    word = splitted[0]
    vectors = asarray(splitted[1:], dtype='float32')
    glove_dict [word] = vectors 


glove.close()

In [53]:
# create dict with only words in out data
word_matrix = zeros((vocab_len, 100))
# iterate throuh all words in our data and find vector for them
for word, index in tokenizer.word_index.items():
    vectors = glove_dict.get(word)
    if vectors is not None:
        word_matrix[index] = vectors # pretrained word embedings with words from our comments
        

# 1. unbalanced data solution, weights 

In [54]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_attack),
                                                 train_attack)

In [55]:
np.unique(train_attack, return_counts=True)

(array([0, 1], dtype=int32), array([78913, 10761]))

In [56]:
weights = {0: class_weights[0], 1: class_weights[1]}
weights

{0: 0.5681826821943152, 1: 4.166620202583403}

# 2. unbalanced data solution, data resampling

In [57]:
np.unique(train_attack, return_counts = True)


(array([0, 1], dtype=int32), array([78913, 10761]))

In [58]:
resampled_train_attack = []
resampled_train_padded = []

y = np.bincount(train_attack)
difference = y[0] - y[1]
num_attacks = 0

for i in range(y[0]+y[1]):
    
    if(train_attack[i]==1):
        resampled_train_padded.append(train_padded[i])
        resampled_train_attack.append(train_attack[i])
    elif(train_attack[i]==0 and num_attacks < y[1]):
        resampled_train_padded.append(train_padded[i])
        resampled_train_attack.append(train_attack[i])
        num_attacks = num_attacks + 1
        
        
resampled_test_attack = []
resampled_test_padded = []

y = np.bincount(test_attack)
difference = y[0] - y[1]
num_attacks = 0

for i in range(y[0]+y[1]):
    
    if(test_attack[i]==1):
        resampled_test_padded.append(test_padded[i])
        resampled_test_attack.append(test_attack[i])
    elif(test_attack[i]==0 and num_attacks < y[1]):
        resampled_test_padded.append(test_padded[i])
        resampled_test_attack.append(test_attack[i])
        num_attacks = num_attacks + 1        

In [59]:
np.asarray(resampled_test_attack).shape

(4890,)

# GRU 

In [ ]:
def GRU_model():
    model = Sequential()
    embedding_layer = Embedding(vocab_len,100,embeddings_initializer = Constant(word_matrix),input_length=longest_sentence_len,trainable=False)
    model.add(embedding_layer)
    model.add(GRU(units = 32, dropout = 0.4))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'], learning_rate = 0.002)
    return model


In [ ]:
model = GRU_model()
model.summary()

In [ ]:
# fitting model with first solution of unbalanced data
 
logdir = "../../logs/GRU_15/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=0,profile_batch=0)

model.fit(train_padded,train_attack,batch_size=64, epochs=15, 
          validation_data=(test_padded,test_attack),verbose=1, shuffle = True, class_weight=weights, callbacks=[tensorboard_callback])

In [ ]:
# fitting model with second solution on unbalanced data

logdir = "../../logs/GRU_16/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=0,profile_batch=0)

model.fit(np.asarray(resampled_train_padded),np.asarray(resampled_train_attack),batch_size=64, epochs=15, 
           validation_data=(np.asarray(resampled_test_padded),np.asarray(resampled_test_attack)),
           verbose=1, shuffle = True, callbacks=[tensorboard_callback])

In [ ]:
model_json = model.to_json()
with open("../../models/GRU_10/model_GRU.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../../models/GRU_10/model_GRU.h5")

# LSTM

In [ ]:
def LSTM_model():
    model = Sequential()
    embedding_layer = Embedding(vocab_len,100,embeddings_initializer = Constant(word_matrix),input_length=longest_sentence_len,trainable=False)
    model.add(embedding_layer)
    model.add(Masking(mask_value=0.0))
    model.add(LSTM(64, return_sequences=False, dropout=0.35))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model2 = LSTM_model()
model2.summary()

In [ ]:
logdir = "../../logs/lstm_1/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=0,profile_batch=0)

lstm_history = model2.fit(train_padded,train_attack,batch_size=128, epochs=15, 
                          validation_data=(test_padded,test_attack),verbose=1, class_weight=weights, 
                          callbacks=[tensorboard_callback])

In [ ]:
model2_json = model2.to_json()
with open("../../logs/lstm_1/model_LSTM.json", "w") as json_file:
    json_file.write(model2_json)
model2.save_weights("../../logs/lstm_1/model_LSTM.h5")

# LSTM with downsampled data model

In [ ]:
def LSTM_downsampled():
    model = Sequential()
    embedding_layer = Embedding(vocab_len,100,embeddings_initializer = Constant(word_matrix),input_length=longest_sentence_len,trainable=False)
    model.add(embedding_layer)
    model.add(Masking(mask_value=0.0))
    model.add(LSTM(16, dropout=0.35))
    model.add(Dense(16,activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model3 = LSTM_downsampled()
model3.summary()

In [ ]:
logdir = "../../logs/lstm_2/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=0,profile_batch=0)

resampled_history = model3.fit(np.asarray(resampled_train_padded),np.asarray(resampled_train_attack),batch_size=64, epochs=15, 
           validation_data=(np.asarray(resampled_test_padded),np.asarray(resampled_test_attack)),
           verbose=1, callbacks=[tensorboard_callback])

In [ ]:
model3_json = model3.to_json()
with open("../../logs/lstm_2/model_LSTM.json", "w") as json_file:
    json_file.write(model3_json)
model3.save_weights("../../logs/lstm_2/model_LSTM.h5")